# Reflection Agent
As the name suggests, *Reflection* is an AI design pattern where the goal is to iteratively refining the output of an LLM generation based on a given task. In practice, a reflection workflow typically involves an LLM generation, followed by a reflection, which in itself is another LLM generation conditioned on the output of the first one. For example, given a task to write an article, the first LLM can generate a draft, and the second LLM can generate a critique of the draft article.

Reflection pattern can be depicted in the following diagram:

<img src="../imgs/reflection-pattern.png" width="600">

The steps shown in the above diagram are defined as followed:

1. User initiates a task rquest. (e.g. Write a blog post)
2. An agent received the request, route to the Generator LLM for the generation task. Optionally, the Generator LLM could use a tool (e.g. knowledge base) to assist in the generation task.
3. The LLM response with an initial draft.
4. Agent evaluate the initial response, then invokes a Reviewer LLM to reflect on the work from the Generation LLM. The Reviewer LLM  critiques the given draft, and response with suggestions for improvements. Optionally, the Reviewer LLM could use a tool to assist in the reviewing process.  
5. Agent receives the response from Reviewer LLM, evaluate the response, either send to the Generator LLM to incorporate the suggestions, or satisfied with the latest version of the work. This process could iterate multiple times until the Reviewer LLM approves the generated work. 
6. Agent returns the final response to the user.


There are 3 agents involved in building this solution: A Generator agent, a Reviewer agent and a supervisor agent that delegates the tasks between these agents. The generator agent will generate a PR document, and the reviewer agent will generate a critique of the PR work done by the generator agent. 

In this lab, we'll implement a reflection based agent using Bedrock Agents. The task is to create a PR for an upcoming release of a movie. 

Let's get started!

## Amazon Bedrock
Amazon Bedrock Agents offers you the ability to build and configure autonomous agents in your application. An agent helps your end-users complete actions based on organization data and user input. Agents orchestrate interactions between foundation models (FMs), data sources, software applications, and user conversations. In addition, agents automatically call APIs to take actions and invoke knowledge bases to supplement information for these actions. 

First, we are going to create a PR generator agent. This agent is responsible for creating draft of PR for an upcoming movie release.
The agent is also integrated with a Knowledge Base that contains the prestine PRs that have been released in the past by the company. The knowledge bases is powered by [Knowledge Bases for Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base.html).

Next, we will create a PR reviewer agent. This agent is responsible for performing a detailed review of a given PR draft generated by the PR generator agent. 

Finally, we'll create a PR supervisor agent which delegrates the responsibility of PR genration and reivew to the appropriate agents. This agent is also responsible for responding to the user once the final version of the PR is complete. 

The following depicts the architecture diagram for this module using Bedrock Agent and Knowledge Bases.

<img src="../imgs/pr-reflection-agent.png" width="600"> 

## Create a Bedrock Agent
On this section we declare global variables that will be act as helpers during entire notebook and you will start to create your first agent.

In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="..")

False

In [3]:
import boto3
import os
import json
import time
from datetime import datetime

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'us.amazon.nova-pro-v1:0'
]

## Importing helper functions
On following section, we're adding bedrock_agent_helper.py and knowledge_base_helper on Python path, so the files can be recognized and their functionalities can be invoked.

In general, the helper functions handle common tasks including agent creation, knowledge bases for bedrock creation and accessing data on S3.

In [5]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
from utils.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

## Create and syncronize Knowledge Base
Before creating an agent, we need to create a Knowledge Base and associates it with the PR generator agent.
This KB will contain some prestine examples of high quality PRs across different movies. We have synthetically generated some example of PRs and stored in the `good_prs` folder. We'll use them as the basis to create our knowledge base system.

This creation process can take several minutes.